In [7]:
import pandas as pd

metrics = pd.read_csv('roberta-base/metrics.csv')

# remove model, accuracy, precision and recall columns
metrics = metrics.drop(columns=['model', 'accuracy', 'precision', 'recall'])

# sort by f1 score
metrics = metrics.sort_values(by='f1', ascending=False)

# remove all rows that are tested on "qi_all"
metrics = metrics[~metrics['tested on'].str.contains('qi_all')]

metrics

,training on,tested on,f1
12,qi_mintaka,qi_mintaka,0.993973
32,qi_all,qi_mintaka,0.993500
22,qi_mintaka_webis_2022_merged,qi_mintaka,0.993046
27,qi_mintaka_webis_2020_merged,qi_mintaka,0.991556
15,qi_webis_2020_webis_2022_merged,qi_webis_2022,0.979757
31,qi_all,qi_webis_2020,0.973936
0,qi_webis_2022,qi_webis_2022,0.973683
26,qi_mintaka_webis_2020_merged,qi_webis_2020,0.969818
20,qi_mintaka_webis_2022_merged,qi_webis_2022,0.968622
17,qi_webis_2020_webis_2022_merged,qi_mintaka,0.968588
